In [ ]:
## Tweet Table

In [4]:
import pymongo
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json

In [5]:
#function to cheack whether 
def check_var(data, x):
    try:
        data[x]
        return True
    except:
        return False

In [6]:
#making mongoDB database and collection
client = MongoClient("localhost", 27017)
db = client['mydatabase']
tweet = db["tweet"]

In [7]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [9]:
file = '/Users/huiwang/Downloads/twitter-out-corona'

tweet.create_index([ ('id', 1) ])

#id, user_id, created_at, text, retweet_count, favorite_count, reply_count, hashtags

import json
with open(file, 'r', encoding = 'utf8') as f1 :
    for line in f1 :
        try :
            data = json.loads(line)
        except :
            continue
        
        hashtag = [] #to collect list of hashtags

        #case 1: check whether it is a retweet data or not
        if (check_var(data, 'retweeted_status')) :
            
            Created_at = pd.to_datetime(data['retweeted_status']['created_at']).strftime('%B %d, %Y, %r')
            UserID = data['retweeted_status']['user']['id']
            ID = data['retweeted_status']['id']
            retweet_count = data['retweeted_status']['retweet_count']
            favorite_count = data['retweeted_status']['favorite_count']
            reply_count = data['retweeted_status']['reply_count']
            
            #check whether there is extension version of that tweet or not        
            if (check_var(data['retweeted_status'], 'extended_tweet')) :
                Text = data['retweeted_status']['extended_tweet']['full_text']
                for t in data['retweeted_status']['extended_tweet']['entities']['hashtags'] :
                    hashtag.append(t['text'])                
            else :
                Text = data['retweeted_status']['text']
                for t in data['retweeted_status']['entities']['hashtags'] :
                    hashtag.append(t['text'])
                    
            #check whether it is retweet of quoted tweet or not
            if (check_var(data['retweeted_status'], 'quoted_status')) :
                
                #check whether there is extension version of the original tweet or not 
                if (check_var(data['retweeted_status']['quoted_status'], 'extended_tweet')) :
                    Text = Text + ' || ' + data['retweeted_status']['quoted_status']['extended_tweet']['full_text']
                    for t in data['retweeted_status']['quoted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else :
                    Text = Text + ' || ' + data['retweeted_status']['quoted_status']['text']
                    for t in data['retweeted_status']['quoted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])               
        
        #case 2: check whether it is a quote tweet data or not
        elif (check_var(data, 'quoted_status')) :
            Created_at = pd.to_datetime(data['created_at']).strftime('%B %d, %Y, %r')
            UserID = data['user']['id']
            ID = data['id']
            retweet_count = data['retweet_count']
            favorite_count = data['favorite_count']
            reply_count = data['reply_count']
            
            #check whether there is extension version of that quote tweet or not 
            if(check_var(data, 'extended_tweet')) :
                Text = data['extended_tweet']['full_text'] + ' || ' 
            else:
                Text = data['text'] + ' || '
            
            for t in data['entities']['hashtags'] :
                hashtag.append(t['text'])
            
            #check whether there is extension version of the originl tweet or not 
            if (check_var(data['quoted_status'], 'extended_tweet')) :
                Text = Text + data['quoted_status']['extended_tweet']['full_text']
                for t in data['quoted_status']['extended_tweet']['entities']['hashtags'] :
                    hashtag.append(t['text'])                
            else:
                Text = Text + data['quoted_status']['text']
                for t in data['quoted_status']['entities']['hashtags'] :
                    hashtag.append(t['text'])
        
        #if it does not belong to case 1 and case 2, it is an ordinary tweet data
        else :
            Created_at = pd.to_datetime(data['created_at']).strftime('%B %d, %Y, %r')
            UserID = data['user']['id']
            ID = data['id']
            retweet_count = data['retweet_count']
            favorite_count = data['favorite_count']
            reply_count = data['reply_count']
            
            #check whether there is extension version of that tweet or not  
            if(check_var(data, 'extended_tweet')) :
                Text = data['extended_tweet']['full_text']
            else :
                Text = data['text']
                
            for t in data['entities']['hashtags'] :
                hashtag.append(t['text'])
        
        tw = {'id' : ID,
              'user_id' : UserID,
              'created_at' : Created_at,
              'text' : Text,
              'retweet_count' : retweet_count,
              'favorite_count' : favorite_count,
              'reply_count' : reply_count,
              'hashtags': hashtag
             }
        
        #if the data is already in our database, we update it 
        if (tweet.count_documents({'id' : tw['id']}) > 0) :
            db.tweet.update_one({'id' : tw['id']},
                                {'$set' : {'retweet_count' : tw['retweet_count'], 'favorite_count' : tw['favorite_count'], 'reply_count' : tw['reply_count']}})
        #if it is not, we insert it    
        else :
            tweet.insert_one(tw)

AttributeError: 'str' object has no attribute 'head'

In [ ]:
## User Table

In [14]:
import pandas as pd
import psycopg2
import json

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
##Creating the database
conn = psycopg2.connect("dbname=postgres port=5432 user=postgres password=password")
conn.set_session(autocommit = True)
cur = conn.cursor()
cur.execute("CREATE DATABASE twitter_users" )
conn.commit()
cur.close()
conn.close()

In [ ]:
#Creating the table
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS users_data( user_id varchar(50) PRIMARY KEY, name varchar(255), user_name varchar(255), verified_status boolean,followers_count integer,friends_count integer,statuses_count integer,user_location varchar(500),favourites_count integer);" )
conn.commit()
cur.close()
conn.close()

In [ ]:
def check_var(data,x):
    try:
        data[x]
        return True
    except:
        return False

In [ ]:
#This function takes the the variable inputs and stores it into a PostgreSQL database
def store_data(user_id, name, user_name, verified_status, followers_count, friends_count,statuses_count,user_location,favourites_count):
    conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
    cur = conn.cursor()
    insert_query = "INSERT INTO users_data (user_id, name, user_name, verified_status, followers_count, friends_count,statuses_count,user_location,favourites_count) VALUES (%s, %s, %s, %s,%s,%s,%s,%s,%s) ON CONFLICT (user_id) DO UPDATE SET verified_status=EXCLUDED.verified_status, followers_count=EXCLUDED.followers_count, friends_count= EXCLUDED.friends_count, statuses_count=EXCLUDED.statuses_count, user_location=EXCLUDED.user_location, favourites_count=EXCLUDED.favourites_count"
    cur.execute(insert_query, (user_id, name, user_name, verified_status, followers_count, friends_count,statuses_count,user_location,favourites_count))
    conn.commit()
    cur.close()
    conn.close()
    
    
# Decoding the JSON file
file = "/Users/varsharajasekar/Downloads/corona-out-2"
with open(file, "r",encoding="utf8") as f1:
    for line in f1:
        
        try:
            data = json.loads(line)
        except:
            continue
        
        
        if (check_var(data,'retweeted_status')):
            
            user_id = data['retweeted_status']['user']['id_str']
            name = data['retweeted_status']['user']['name']
            user_name = data['retweeted_status']['user']['screen_name']
            verified_status = data['retweeted_status']['user']['verified']
            followers_count = data['retweeted_status']['user']['followers_count']
            friends_count = data['retweeted_status']['user']['friends_count']
            statuses_count = data['retweeted_status']['user']['statuses_count']
            user_location = data['retweeted_status']['user']['location']
            favourites_count= data['retweeted_status']['user']['favourites_count']
            
           
        else :
            user_id = data['user']['id_str']
            name = data['user']['name']
            user_name = data['user']['screen_name']
            verified_status = data['user']['verified']
            followers_count = data['user']['followers_count']
            friends_count = data['user']['friends_count']
            statuses_count = data['user']['statuses_count']
            user_location = data['user']['location']
            favourites_count= data['user']['favourites_count']
            
               
        #insert the data into the PostgreSQL database
        store_data(user_id, name, user_name, verified_status, followers_count, friends_count,statuses_count,user_location,favourites_count)

In [ ]:
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()


# A function that takes in a PostgreSQL query and outputs a pandas database 
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [ ]:
create_pandas_table("SELECT * FROM users_data ")